# DB stats pre-process
Every hour, refresh the response for the `/stats` endpoint so as to make it almous instantaneous.

Include `last_updated` value for the UI.

Saves a JSON that can directly be returned by the api.

In [ ]:
# required imports to access api_db, misc, misc.CONFIG, ...
import sys
sys.path = ['.', '..', '../..'] + sys.path
from collection import *

<hr>
<h1 align="center">driver code</h1>

In [ ]:
from datetime import datetime, date, timedelta

In [ ]:
# aggregate faceted metrics for users
one_month_ago = datetime.now() - timedelta(days=31)
faceted_users = next(api_db.col_users.aggregate([
    {"$facet": {
        "suspended": [
            {"$match": {"suspended": True}},
            {"$count": "suspended"},
        ],
        "private": [
            {"$match": {"private": True}},
            {"$count": "private"}
        ],
        "notfound": [
            {"$match": {"notfound": True}},
            {"$count": "notfound"}
        ] #,
        # "suspended_last_week": [
        #     {"$match": {"suspended": True, "time_suspended": {"$gte": datetime.now() - timedelta(days=7)}}},
        #     {"$count": "suspended_last_week"},
        # ],
        # "suspended_last_month": [
        #     {"$match": {"suspended": True, "time_suspended": {"$gte": datetime.now() - timedelta(days=31)}}},
        #     {"$count": "suspended_last_month"},
        # ],
        # "created_last_week": [
        #     {"$match": {"created_at": {"$gte": datetime.now() - timedelta(days=7)}}},
        #     {"$count": "created_last_week"}
        # ],
        # "created_last_month": [
        #     {"$match": {"created_at": {"$gte": datetime.now() - timedelta(days=31)}}},
        #     {"$count": "created_last_month"}
        # ],
        # "created_last_6_months": [
        #     {"$match": {"created_at": {"$gte": datetime.now() - timedelta(days=31*6)}}},
        #     {"$count": "created_last_6_months"}
        # ],
        # "created_since_september": [
        #     {"$match": {"created_at": {"$gte": datetime(2020, 9, 1)}}},
        #     {"$count": "created_since_september"}
        # ]
    }},
    {"$project": {
        "suspended": {"$arrayElemAt": ["$suspended.suspended", 0]},
        "private": {"$arrayElemAt": ["$private.private", 0]},
        "notfound": {"$arrayElemAt": ["$notfound.notfound", 0]} #,
        # "suspended last week": {"$arrayElemAt": ["$suspended_last_week.suspended_last_week", 0]},
        # "suspended last month": {"$arrayElemAt": ["$suspended_last_month.suspended_last_month", 0]},
        # "created last week": {"$arrayElemAt": ["$created_last_week.created_last_week", 0]},
        # "created last month": {"$arrayElemAt": ["$created_last_month.created_last_month", 0]},
        # "created last 6 months": {"$arrayElemAt": ["$created_last_6_months.created_last_6_months", 0]},
        # "created since september 2020": {"$arrayElemAt": ["$created_since_september.created_since_september", 0]}
    }}
]))
print(faceted_users)

In [ ]:
# aggregate faceted metrics for tweets
faceted_tweets = next(api_db.col_tweets.aggregate([
    {"$facet": {
        "retweets": [
            {"$match": {"retweeted_status": {"$exists": True}}},
            {"$count": "retweets"},
        ],
        "quotes": [
            {"$match": {"quoted_status": {"$exists": True}}},
            {"$count": "quotes"},
        ],
        "replies": [
            {"$match": {"in_reply_to_status_id": {"$exists": True}}},
            {"$count": "replies"},
        ]
    }},
    {"$project": {
        "retweets": {"$arrayElemAt": ["$retweets.retweets", 0]},
        "quotes": {"$arrayElemAt": ["$quotes.quotes", 0]},
        "replies": {"$arrayElemAt": ["$replies.replies", 0]}
    }}
]))
print(faceted_tweets)

In [ ]:
res = {
    "users": {
        "total": api_db.col_users.estimated_document_count()
    },
    "tweets": {
        "total": api_db.col_tweets.estimated_document_count()
    },
    "mb": int(api_db.db.command("dbstats")["dataSize"] / (1024 * 1024))
}
print(res)

In [ ]:
res["users"].update(faceted_users)
res["tweets"].update(faceted_tweets)
tweet_types = ["retweets", "quotes", "replies"]
if all(x in res["tweets"] for x in tweet_types):
    res["tweets"]["original"] = res["tweets"]["total"] - sum(res["tweets"][x] for x in tweet_types)
print(res)

In [ ]:
import json

In [ ]:
db_stats_api = abs_path("..") + os.sep + misc.CONFIG["database_stats_file_api"]

In [ ]:
with open(db_stats_api, "w", encoding="utf-8") as outf:
    json.dump(res, outf)

In [ ]:
print("DONE")